<a href="https://colab.research.google.com/github/OllyMuh/Python_for_DataScience/blob/Real_Estate_Prices_Project/EDA_RealEstate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import pandas as pd
import numpy as np
import matplotlib

import matplotlib.image as img
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats

%matplotlib inline

# отключаем предупреждения
import warnings
warnings.filterwarnings('ignore')

# пути к директориям и файлам
DATASET_PATH = '/content/drive/MyDrive/train_prepared.csv'
PREPARED_DATASET_PATH = '/content/drive/MyDrive/train_prepared_EDA.csv'

In [13]:
matplotlib.rcParams.update({'font.size': 14})

In [3]:
df = pd.read_csv(DATASET_PATH, sep=',')

**Оптимизируем объем используемой памяти** 

In [5]:
def reduce_mem_usage(df):
  start_mem = df.memory_usage().sum() / 1024 ** 2
  print(f'Memory usage is {start_mem} Mb')

  for col in df.columns:
    col_type = df[col].dtype

    if col_type != object:
      c_min = df[col].min()
      c_max = df[col].max()
      if str(col_type)[:3] == 'int':
        if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
          df[col] = df[col].astype(np.int8)
        elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
          df[col] = df[col].astype(np.int16)
        elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
          df[col] = df[col].astype(np.int32)
        elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
          df[col] = df[col].astype(np.int64)
      else:
        if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
          df[col] = df[col].astype(np.float32)
        else:
          df[col] = df[col].astype(np.float64)
    else:
      df[col] = df[col].astype('category')
  end_mem = df.memory_usage().sum() / 1024 **2
  print(f'Memory usage after {end_mem} Mb')
  return df

In [ ]:
# reduce_mem_usage(df)
df.dtypes